### Data Management

In [446]:
# Importing Files and packages

import pandas as pd
import numpy as np

Player_stats = pd.read_csv("NBA Player Stats 2023 RS.csv")
Player_salaries = pd.read_excel("NBA Players Salary 2023.xlsx")
Team_stats = pd.read_excel("NBA Team Stats.xlsx")

In [447]:
# Merging datasets of salary and key statistics
Players = Player_stats.merge(Player_salaries, how="inner", on=["NAME", "TEAM"])

# Make 2point and 3point attempts a per game statistic
Players["2PApg"] = Players["2PA"]/Players["GP"]
Players["3PApg"] = Players["3PA"]/Players["GP"]

# Removing Variable columns that wont be used for Optimization
Players = Players.drop(["RANK", "AGE", "GP", "USG%", "TO%", "FTA", "2PA", "3PA", "eFG%", "TS%", 
                        "TPG", "P+R", "P+A", "P+R+A", "VI", "ORtg", "DRtg"], axis=1)

# Changing Categorical Variable names
Players["POS"] = Players["POS"].replace({"C-F":"CF", "F-G":"FG", "G-F":"FG", "F-C":"CF"})

In [448]:
# Changing position variable to binary to easier compute the optimization
Players['Guard'] = np.where(np.logical_or((Players['POS']== "G"), (Players["POS"]=="FG")), 1, 0)

Players['F'] = np.where(np.logical_or((Players['POS']== "CF"), (Players["POS"]=="FG")), 1, 0)
Players['Forward'] = np.where(np.logical_or((Players['POS']== "F"), (Players["F"]== 1)), 1, 0)

Players['Center'] = np.where(np.logical_or((Players['POS']== "C"), (Players["POS"]=="CF")), 1, 0)
Players = Players.drop(["POS", "F"], axis=1)

Players["Player"] = 1

Player_ID = []
for i in range(len(Players)):
    Player_ID.append("x_"+str(i+1))
Players["Player_ID"] = Player_ID

In [449]:
# Renaming columns and removing columns in dataset for the championship teams
Players = Players.rename(columns = {"PPG":"PTS", "RPG":"REB", "APG":"AST", "SPG":"STL","BPG":"BLK","TEAM_y":"Team","2PApg":"2PA", "3PApg":"3PA"})
Team_stats = Team_stats.rename(columns = {"TRB":"REB"})
Teams = Team_stats.drop(["FGA", "FG%", "ORB", "DRB"], axis=1)

## Optimization

In [450]:
from scipy.optimize import LinearConstraint
from scipy.optimize import minimize
from pulp import *
import openpyxl
import re

In [451]:
# Function for multiplying column statistic with decision variable
import operator
def dotproduct(vec1, vec2):
    return sum(map(operator.mul, vec1, vec2))

In [452]:
# Creating Decisiom Variables
dec_var = []
for i in range(len(Players)):
     var_name = "x_"+str(i+1)
     var_name = LpVariable(var_name, cat="Binary")
     dec_var.append(var_name)

# Creating the optimization problem
min_salary = LpProblem("min_salary", LpMinimize) 

# Defining Objective Function
min_salary += dotproduct(Players["Salary"], dec_var)

In [453]:
# Adding sum statistical constraints
constraint_var = ["3PA", "2PA", "REB", "AST", "STL", "BLK", "PTS"]
for col in constraint_var:
    min_salary += dotproduct(Players[col], dec_var) >= Teams[col].values[-1]

# Adding average statistical constraints
constraint_var = ["3P%", "2P%", "FT%"]
for col in constraint_var:
    min_salary += dotproduct(Players[col], dec_var)/15 >= Teams[col].values[-1]

# Adding total roster constraints
min_salary += dotproduct(Players["Player"], dec_var) == 15

# Adding Positional constraints
posistions = ["Guard", "Forward", "Center", "All Star"]
pos_constraints = [5, 5, 3, 1]

for i in range(0,len(posistions)):
    min_salary += dotproduct(Players[posistions[i]], dec_var) >= pos_constraints[i]

In [454]:
# Optimizing solution
res = min_salary.solve()


In [455]:
# Gathering row indexes for the selected players
indexes = []
for element in dec_var:
    if element.varValue == 1:
        player_index = Players.index[Players["Player_ID"] == str(element)].tolist()
        indexes.append(player_index)

In [456]:
# Creating dataframe from the rows of all the optimal players
optimal_team_rows = []
for x in indexes:
    for y in x:
        row = Players.iloc[y]
        optimal_team_rows.append(row) 

optimal_team = pd.DataFrame(optimal_team_rows, columns=Players.columns)
optimal_team

,NAME,TEAM,MPG,FT%,2P%,3P%,PTS,REB,AST,STL,BLK,Salary,All Star,2PA,3PA,Guard,Forward,Center,Player,Player_ID
30,Desmond Bane,Mem,31.8,0.883,0.534,0.408,21.5,5.0,4.4,1.0,0.4,2130240,0,9.172414,7.017241,1,0,0,1,x_31
36,Tyrese Haliburton,Ind,33.6,0.867,0.572,0.400,20.7,3.7,10.4,1.6,0.4,4215120,1,7.839286,7.178571,1,0,0,1,x_37
92,Kris Dunn,Uta,25.8,0.774,0.550,0.472,13.2,4.5,5.6,1.1,0.5,735819,0,8.181818,1.636364,1,0,0,1,x_93
97,Austin Reaves,Lal,28.8,0.864,0.631,0.398,13.0,3.0,3.4,0.5,0.3,1563518,0,4.281250,3.375000,1,0,0,1,x_98
113,Naz Reid,Min,18.4,0.677,0.653,0.346,11.5,4.9,1.1,0.6,0.8,1930681,0,5.294118,3.191176,0,1,1,1,x_114
145,Eugene Omoruyi,Det,21.9,0.723,0.526,0.293,9.7,3.5,1.0,0.8,0.2,555402,0,4.470588,3.411765,0,1,0,1,x_146
219,Drew Eubanks,Por,20.3,0.664,0.655,0.389,6.6,5.4,1.3,0.5,1.3,1836090,0,4.051282,0.230769,0,1,1,1,x_220
220,Jordan Goodwin,Was,17.8,0.768,0.511,0.322,6.6,3.3,2.7,0.9,0.4,900000,0,3.790323,1.903226,1,0,0,1,x_221
276,Meyers Leonard,Mil,12.6,0.889,0.636,0.389,4.8,3.8,0.1,0.2,0.0,105522,0,1.222222,2.000000,0,1,1,1,x_277
287,Damian Jones,Uta,15.8,0.778,0.714,0.714,4.6,3.5,0.6,0.3,0.5,2298385,0,1.842105,0.736842,0,0,1,1,x_288


In [457]:
optimal_team_sum = optimal_team[["PTS", "REB", "AST", "STL", "BLK", "2PA", "3PA"]].sum()
optimal_team_sum

PTS    130.900000
REB     48.300000
AST     35.300000
STL      9.200000
BLK      6.000000
2PA     58.378417
3PA     37.571431
dtype: float64

How to Improve the model:
- Improve the all star metric to a more advanced or "bullet proof" metric.
- Add some type of chemistry constraint; in order to account for language barriers or cultural backgrounds. Should have impact on team performance.
- Add playoff experience or leauge experience as a metric; in order to account more for the difference in a playoff series to regular season games.
- Make salaries more realistic; account for the overperforming newbees or newly traded and unfitted palyers. This would be difficult and change the premis some, as we would look at the changes in "Signing" the player to said wage by their future wages.

### Changing per game variables to per minute to better measure a players actual impact

In [458]:
per_game_var = ["PTS", "REB", "AST", "STL", "BLK", "2PA", "3PA"]

Players2 = Players

for col in Players2:
    if col in per_game_var:
        Players2[col] = Players2[col]/Players2["MPG"]

per_game_var = ["PTS", "REB", "AST", "STL", "BLK", "2PA", "3PA"]

Teams2 = Teams

for col in Teams2:
    if col in per_game_var:
        Teams2[col] = Teams2[col]/48

In [459]:
# Creating Decisiom Variables
dec_var = []
for i in range(len(Players)):
     var_name = "x_"+str(i+1)
     var_name = LpVariable(var_name, cat="Binary")
     dec_var.append(var_name)

# Creating the optimization problem
min_salary = LpProblem("min_salary", LpMinimize)

# Defining Objective Function
min_salary += dotproduct(Players["Salary"], dec_var)

In [460]:
# Adding sum statistical constraints
constraint_var = ["3PA", "2PA", "REB", "AST", "STL", "BLK", "PTS"]
for col in constraint_var:
    min_salary += dotproduct(Players[col], dec_var) >= Teams[col].values[-1]

# Adding average statistical constraints
constraint_var = ["3P%", "2P%", "FT%"]
for col in constraint_var:
    min_salary += dotproduct(Players[col], dec_var)/15 >= Teams[col].values[-1]

# Adding total roster constraints
min_salary += dotproduct(Players["Player"], dec_var) == 15

# Adding Positional constraints
posistions = ["Guard", "Forward", "Center", "All Star"]
pos_constraints = [5, 5, 3, 1]

for i in range(0,len(posistions)):
    min_salary += dotproduct(Players[posistions[i]], dec_var) >= pos_constraints[i]

In [461]:
# Optimizing solution
res = min_salary.solve()

In [462]:
# Gathering row indexes for the selected players
indexes = []
for element in dec_var:
    if element.varValue == 1:
        player_index = Players.index[Players["Player_ID"] == str(element)].tolist()
        indexes.append(player_index)


In [463]:
# Creating dataframe from the rows of all the optimal players
optimal_team_rows = []
for x in indexes:
    for y in x:
        row = Players.iloc[y]
        optimal_team_rows.append(row) 

optimal_team = pd.DataFrame(optimal_team_rows, columns=Players.columns)
optimal_team

,NAME,TEAM,MPG,FT%,2P%,3P%,PTS,REB,AST,STL,BLK,Salary,All Star,2PA,3PA,Guard,Forward,Center,Player,Player_ID
36,Tyrese Haliburton,Ind,33.6,0.867,0.572,0.400,0.616071,0.110119,0.309524,0.047619,0.011905,4215120,1,0.233312,0.213648,1,0,0,1,x_37
92,Kris Dunn,Uta,25.8,0.774,0.550,0.472,0.511628,0.174419,0.217054,0.042636,0.019380,735819,0,0.317125,0.063425,1,0,0,1,x_93
169,Jaden Hardy,Dal,14.8,0.823,0.469,0.404,0.594595,0.128378,0.094595,0.027027,0.006757,1017781,0,0.249155,0.219595,1,0,0,1,x_170
220,Jordan Goodwin,Was,17.8,0.768,0.511,0.322,0.370787,0.185393,0.151685,0.050562,0.022472,900000,0,0.212939,0.106923,1,0,0,1,x_221
225,Cody Zeller,Mia,14.4,0.686,0.649,0.000,0.451389,0.298611,0.048611,0.013889,0.020833,517060,0,0.263889,0.009259,0,1,1,1,x_226
271,Sam Merrill,Cle,11.7,1.000,1.000,0.278,0.427350,0.153846,0.085470,0.068376,0.000000,1000000,0,0.068376,0.307692,1,0,0,1,x_272
276,Meyers Leonard,Mil,12.6,0.889,0.636,0.389,0.380952,0.301587,0.007937,0.015873,0.000000,105522,0,0.097002,0.158730,0,1,1,1,x_277
298,Admiral Schofield,Orl,12.2,0.913,0.646,0.324,0.344262,0.139344,0.065574,0.016393,0.008197,506508,0,0.106336,0.163934,0,1,0,1,x_299
299,Ish Wainright,Pho,15.3,0.839,0.500,0.329,0.274510,0.150327,0.058824,0.039216,0.026144,474851,0,0.058824,0.188453,0,1,0,1,x_300
305,Terry Taylor,Chi,7.1,0.250,0.889,1.000,0.563380,0.225352,0.000000,0.000000,0.028169,1563518,0,0.253521,0.028169,1,0,0,1,x_306


In [464]:
optimal_team_sum = optimal_team[["PTS", "REB", "AST", "STL", "BLK", "2PA", "3PA"]].sum()*48
optimal_team_sum

PTS    304.873020
REB    131.738340
AST     65.012665
STL     21.711808
BLK     12.723144
2PA    138.474871
3PA     88.986425
dtype: float64